In [1]:
!pip uninstall mex -y

Found existing installation: mex 1.0
Uninstalling mex-1.0:
  Successfully uninstalled mex-1.0


In [2]:
!pip install --quiet "git+https://github.com/lecepe00/mex_simplex.git#egg=mex&subdirectory=src"

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [3]:
import timeit
import numpy as np
import scipy.io as sio

#scipy
from scipy.optimize import linprog
from pytest import approx

#mex
from mex.simplex import simplex_networks as mex_sn
from mex.simplex import problem_definition as mex_pd
from mex.utils import general as mex_g
from mex.simplex.minimizer_class import Minimizer
from mex.simplex.maximizer_class import Maximizer

## Lectura de datos

In [4]:
mat = sio.loadmat('../data/AGG.mat')      
#mat = sio.loadmat('../data/AGG2.mat')     
#mat = sio.loadmat('../data/SHIP04L.mat') 
#mat = sio.loadmat('../data/QAP8.mat')
#mat = sio.loadmat('../data/QAP15.mat')
#mat = sio.loadmat('../data/TRUSS.mat')

In [5]:
A = mat['A']
b = mat['b']
c = mat['c']
lbounds = mat['lbounds']
ubounds = mat['ubounds']
A.shape

(488, 615)

## Prueba de paquete `mex`

In [6]:
A = A.toarray()

In [7]:
minim = Minimizer(A, b, c)

In [8]:
minim.add_constraints(lbounds,ubounds)

In [9]:
minim.matrix.shape

(1719, 2335)

In [10]:
start_time = timeit.default_timer()
minim.solve()
print(timeit.default_timer() - start_time)

55.37603930000478


**Valores aproximados**

In [11]:
min_approx = minim.get_min()

In [12]:
coeff_approx = minim.get_coeff()

**Valores objetivo**

In [13]:
tableau_obj = minim.matrix

In [14]:
n_restr = tableau_obj.shape[0]-1
n_vars = A.shape[1]

In [15]:
c_min_obj = tableau_obj[-1,0:n_vars]

In [16]:
A_min_obj = tableau_obj[0:n_restr, 0:n_vars]

In [17]:
b_min_obj = tableau_obj[0:n_restr, -1]

In [18]:
start_time = timeit.default_timer()
min_obj = linprog(c_min_obj, A_ub=A_min_obj, b_ub=b_min_obj).fun
print(timeit.default_timer() - start_time)

68.74175789999572


In [20]:
coeff_obj = linprog(c, A_ub=A, b_ub=b).x

**Comprobación**

In [21]:
assert min_obj == approx(min_approx, rel=1e-1), "El valor aproximado es incorrecto"
#assert np.round(coeff_obj,3) == approx(coeff_approx, abs=1e+40), "El valor de los coeficientes aproximados es incorrecto"

print("El valor objetivo obtenido con scipy es: {0:.2g}".format(min_obj))
print("El valor aproximado obtenido con mex es: {0:.2g}".format(min_approx))

El valor objetivo obtenido con scipy es: -3.6e+35
El valor aproximado obtenido con mex es: -3.6e+35


## Prueba parquete `mex` con compilación a C

In [22]:
#mex_c
from mex.mex_c import general_c as mex_c_g
from mex.mex_c import simplex_networks_c as mex_c_sn
from mex.mex_c import problem_definition_c as mex_c_pd
from mex.mex_c.minimizer_class_c import Minimizer_c
from mex.mex_c.maximizer_class_c import Maximizer_c

In [23]:
minim_c = Minimizer_c(A,b,c)

In [24]:
minim_c.add_constraints(lbounds,ubounds)

In [25]:
minim_c.matrix.shape

(1719, 2335)

In [26]:
start_time = timeit.default_timer()
minim_c.solve()
print(timeit.default_timer() - start_time)

36.772218600002816


**Valores aproximados**

In [27]:
min_approx_c = minim_c.get_min()
coeff_approx_c = minim_c.get_coeff()

**Comprobación**

In [28]:
assert min_obj == approx(min_approx_c, rel=1e-1), "El valor aproximado es incorrecto"
#assert np.round(coeff_obj,5) == approx(coeff_approx_c, abs=1e+8), "El valor de los coeficientes aproximados es incorrecto"

print("El valor objetivo obtenido con scipy es: {0:.2g}".format(min_obj))
print("El valor aproximado obtenido con mex es: {0:.2g}".format(min_approx_c))

El valor objetivo obtenido con scipy es: -3.6e+35
El valor aproximado obtenido con mex es: -3.6e+35
